In [1]:
%load_ext autoreload
%autoreload 2
import pickle
import os
os.chdir('/home/mjc/github/EHRVis/')
with open('data/batches/val/2014_104.pckl','rb') as f:
    samples = pickle.load(f)

In [2]:
import torch
from torch import nn,optim
from torch.autograd import Variable
import torch.nn.functional as F
import time
from functions import date_converter

In [ ]:
os.curdir

In [3]:
inputs = Variable(torch.Tensor(100,50,128).normal_())
# inputs = Variable(torch.Tensor(100,100,128).normal_().cuda())

In [4]:
# try opening custom RNN
from models.time_rnn import rnn_base, rnn_functions

In [ ]:
start = time.time()
out = date_converter(20140131)
print(time.time()-start)

In [11]:
class Test(nn.Module):
    def __init__(self):
        super(Test,self).__init__()
        self.hidden_size = 128
        self.input_size = 128
        self.output_size = 268
    
        self.RNN1 = nn.LSTM(hidden_size=self.hidden_size, input_size=self.input_size, bias=False, batch_first=True)
        self.RNN2 = rnn_base.LSTM(hidden_size=self.hidden_size, input_size=self.input_size, bias=False, batch_first=True)
        self.RNN3 = nn.LSTMCell(hidden_size=self.hidden_size, input_size=self.input_size, bias=False)
        self.RNN4 = rnn_base.LSTMCell_time(hidden_size=self.hidden_size, input_size=self.input_size, bias=False)
        
    def forward(self,inputs):
        start = time.time()
        b, seq, hid = inputs.size()

        start = time.time()
        hid1, cell1 = self.RNN1(inputs)[1]
        print("Method 1")
        print(time.time()-start)
        start = time.time()
        hid2, cell2 = self.RNN2(inputs)[1]
        print("Method 2")
        print(time.time()-start)
        start = time.time()
        hid3 = Variable(torch.randn(b,hid).normal_())
        cell3 = Variable(torch.Tensor(b,hid).normal_())
        for i in range(seq):
            hid3, cell3 = self.RNN3(inputs[:,i,:],(hid3,cell3))
        print("Method 3")
        print(time.time()-start)

        start = time.time()
        hid4 = Variable(torch.randn(b,hid).normal_())
        cell4 = Variable(torch.Tensor(b,hid).normal_())
        for i in range(seq):
            hid4, cell4 = self.RNN3(inputs[:,i,:],(hid4,cell4))
        print("Method 4")
        print(time.time()-start)
#         return ((hid3,cell3),(hid4,cell4))
#         return ((hid1,cell1),(hid2,cell2),(hid3,cell3))

In [14]:
test= Test()
test.cpu()
# test.cuda()

Test(
  (RNN1): LSTM(128, 128, bias=False, batch_first=True)
  (RNN2): LSTM(128, 128, bias=False, batch_first=True)
  (RNN3): LSTMCell(128, 128, bias=False)
  (RNN4): LSTMCell_time(128, 128, bias=False)
)

In [15]:
test(inputs)

Method 1
0.14815497398376465
([[Parameter containing:
-8.5452e-02 -2.1052e-02  8.7528e-02  ...  -3.7807e-02  7.6631e-02  4.2297e-02
 3.2834e-02  7.2931e-02 -8.2386e-02  ...   8.2221e-03 -6.0625e-02 -6.9933e-02
-8.2728e-02  8.3201e-02  7.2076e-02  ...   1.4865e-02  6.1393e-02 -1.5973e-02
                ...                   ⋱                   ...                
-5.6404e-02  6.8817e-02 -2.2709e-02  ...  -2.7402e-02 -5.0894e-03 -4.9341e-02
 5.8236e-02  7.0276e-03  7.8666e-02  ...   1.5080e-03  5.0784e-02  6.2848e-02
-9.3417e-03 -6.5837e-02 -5.0387e-02  ...  -1.4167e-02 -3.3260e-02 -5.7781e-02
[torch.FloatTensor of size 512x128]
, Parameter containing:
1.00000e-02 *
 7.1524 -0.2093 -4.0871  ...   7.0079 -0.9767 -4.0853
 8.5470 -7.4804 -7.5928  ...  -7.1755 -0.8471  8.4880
 6.6776 -0.8398 -5.5200  ...   8.7501  2.5887 -0.4949
          ...             ⋱             ...          
 5.6955 -1.0198 -4.2755  ...   7.5038  6.7103  5.8915
 5.8921 -8.8312  6.1982  ...   1.8603  8.0238  3.1309
 6

TypeError: LSTMCell_time() missing 2 required positional arguments: 'w_hh' and 'w_d'

1. w/ cuda
  1. 0.14, 0.0003~0.001 for each iter
  2. 0.006
2. w/o cuda
  1. 0.09~0.2
  2. 0.09~0.2

In [16]:
help(nn.LSTMCell)

Help on class LSTMCell in module torch.nn.modules.rnn:

class LSTMCell(RNNCellBase)
 |  A long short-term memory (LSTM) cell.
 |  
 |  .. math::
 |  
 |      \begin{array}{ll}
 |      i = \mathrm{sigmoid}(W_{ii} x + b_{ii} + W_{hi} h + b_{hi}) \\
 |      f = \mathrm{sigmoid}(W_{if} x + b_{if} + W_{hf} h + b_{hf}) \\
 |      g = \tanh(W_{ig} x + b_{ig} + W_{hc} h + b_{hg}) \\
 |      o = \mathrm{sigmoid}(W_{io} x + b_{io} + W_{ho} h + b_{ho}) \\
 |      c' = f * c + i * g \\
 |      h' = o * \tanh(c') \\
 |      \end{array}
 |  
 |  Args:
 |      input_size: The number of expected features in the input x
 |      hidden_size: The number of features in the hidden state h
 |      bias: If `False`, then the layer does not use bias weights `b_ih` and
 |          `b_hh`. Default: ``True``
 |  
 |  Inputs: input, (h_0, c_0)
 |      - **input** (batch, input_size): tensor containing input features
 |      - **h_0** (batch, hidden_size): tensor containing the initial hidden
 |        state for e

In [ ]:
cell = Variable(torch.randn(100,128))

In [43]:
x = Variable(torch.randn(100,128))
h = Variable(torch.randn(100,128))

# method 1
start = time.time()
W_all = nn.Linear(128,128*4)
U_all = nn.Linear(128,128*4)
f,i,o,c = torch.chunk(W_all(x)+U_all(h),4,1)
print(time.time()-start)

# method 2
start = time.time()
W_i = nn.Linear(128,128)
W_f = nn.Linear(128,128)
W_o = nn.Linear(128,128)
W_c = nn.Linear(128,128)
U_i = nn.Linear(128,128)
U_f = nn.Linear(128,128)
U_o = nn.Linear(128,128)
U_c = nn.Linear(128,128)
i = W_i(x)+U_i(h)
o = W_o(x)+U_o(h)
f = W_f(x)+U_f(h)
c = W_c(x)+U_c(h)
print(time.time()-start)


0.018282175064086914
0.0058746337890625


In [48]:
out_list = []
W = nn.Linear(4,4)
hx = Variable(torch.randn(2,4))
for i in range(5):
    out_list.append(hx)
    hx = W(hx)


In [53]:
input_size = 128
hidden_size = 128
W_all = nn.Linear(input_size, hidden_size*4)
U_all = nn.Linear(hidden_size, hidden_size*4, bias=False)
W_d = nn.Linear(hidden_size, hidden_size)

In [79]:
b = 500
x = Variable(torch.randn(b,hidden_size), requires_grad=False)
h = Variable(torch.randn(b,hidden_size), requires_grad=False)
c = Variable(torch.randn(b,hidden_size), requires_grad=False)
x = x.cpu()
h = h.cpu()
c = c.cpu()

W_all.cuda()
U_all.cuda()
W_d.cuda()
start = time.time()
outputs = []
x = x.cuda()
h = h.cuda()
c = c.cuda()
for i in range(100):
    c_s1 = F.tanh(W_d(c))
    c_s2 = c_s1 * 0.36
    c_l = c - c_s1
    c_adj = c_l + c_s2
    outs = W_all(h)+U_all(x)
    f, i, o, c_tmp = torch.chunk(outs,4,1)
    c = f*c_adj + i*c_tmp
    h = o*F.tanh(c)
    outputs.append(h)
outputs = torch.stack(outputs,1)
print(time.time()-start)

0.1501924991607666


In [78]:
torch.stack(outputs,1)

Variable containing:
( 0 ,.,.) = 
 -3.4575e-01  2.0295e-01  6.5576e-04  ...  -5.4149e-01  5.6244e-01  1.7737e-01
 -8.3154e-02  7.5503e-01  1.7961e-02  ...   5.5867e-02 -1.3854e-02 -1.1016e-01
  1.7480e-01  1.9724e-01 -1.6898e-03  ...  -4.5279e-02 -3.5027e-02  1.0929e-01
                 ...                   ⋱                   ...                
 -8.9460e-04  5.0247e-01  1.8987e-03  ...   1.9689e-01 -1.1313e-01 -1.8779e-01
  1.2379e-01  2.7645e-01 -5.8429e-02  ...  -8.2195e-02 -9.7530e-02 -2.2976e-01
 -8.8876e-04  5.0248e-01  1.8993e-03  ...   1.9689e-01 -1.1314e-01 -1.8778e-01

( 1 ,.,.) = 
 -1.0946e-02  1.3321e-01  1.0045e-01  ...   2.7296e-03  1.3434e-01  8.4473e-02
  2.0708e-01  2.6728e-01  2.3116e-01  ...  -2.5411e-01 -1.5995e-01  4.0877e-02
  3.0244e-01 -9.9020e-02  2.0334e-01  ...   1.6241e-01  6.0920e-02  5.0653e-02
                 ...                   ⋱                   ...                
  2.5592e-01  3.9311e-01  5.0377e-02  ...  -1.3236e-01 -1.8280e-01 -2.6516e-02
  1.